In [2]:
print(df.columns)

Index(['barcode', 'Celltypist:Fetal_Human_Skin:predicted_labels',
       'pct_counts_gene_group__mito_transcript', 'qc.filter.pass',
       'Azimuth:predicted.celltype.l3', 'vireo.donor.id',
       'DoubletFinder.score', 'Azimuth:predicted.celltype.l2.score',
       'Celltypist:Adult_Human_Skin:over_clustering', 'scDblFinder.score',
       'Celltypist:Immune_All_High:over_clustering',
       'Azimuth:mapping.score.celltype.l3', 'cell_passes_hard_filters',
       'cell_passes_qc-MAD-3,-3,3-per:all_together:score',
       'pct_counts_in_top_50_genes',
       'total_counts_gene_group__mito_transcript',
       'Azimuth:predicted.celltype.l1', 'tranche.id', 'experiment.id',
       'scds.multiplet', 'Celltypist:Immune_All_Low:over_clustering',
       'Celltypist:Immune_All_High:conf_score',
       'Celltypist:Adult_Human_Skin:majority_voting',
       'Celltypist:Fetal_Human_Skin:majority_voting',
       'Celltypist:Immune_All_Low:majority_voting', 'scrublet.multiplet',
       'Celltypist:Adu

In [1]:
import os
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

# Define the donor IDs (adjusted to match the file names)
donor_ids = ['HRX138216', 'HRX138217', 'HRX138218', 'HRX138219', 'HRX138220', 'HRX138221', 'HRX138222', 'HRX138223', 'HRX138224']

# Define the metadata for donor ages, including the age group and specific age
donor_metadata = {
    'HRX138216': {'age_group': 'Young', 'age': 18},
    'HRX138217': {'age_group': 'Young', 'age': 22},
    'HRX138218': {'age_group': 'Young', 'age': 23},
    'HRX138219': {'age_group': 'Middle', 'age': 44},
    'HRX138220': {'age_group': 'Middle', 'age': 47},
    'HRX138221': {'age_group': 'Middle', 'age': 48},
    'HRX138222': {'age_group': 'Older', 'age': 70},
    'HRX138223': {'age_group': 'Older', 'age': 73},
    'HRX138224': {'age_group': 'Older', 'age': 76}
}

# Directory paths for .h5ad and .tsv files
h5ad_directory = '/mnt/bmh01-rds/Eckersley_Lab/NiallGarvey'
tsv_directory = '/mnt/bmh01-rds/Eckersley_Lab/NiallGarvey'

# Process each donor ID one by one
for donor_id in donor_ids:
    print(f"Processing {donor_id}...")

    # Process .h5ad file
    h5ad_file = os.path.join(h5ad_directory, f'{donor_id}.donor.h5ad')
    if os.path.exists(h5ad_file):
        adata = sc.read(h5ad_file)
        adata.obs_names_make_unique()

        # Add donor age group and specific age to the AnnData object
        donor_info = donor_metadata.get(donor_id, {'age_group': 'Unknown', 'age': None})
        adata.obs['donor_age_group'] = donor_info['age_group']
        adata.obs['donor_age'] = donor_info['age']

        # Check if adata.X is populated and has the correct shape
        print(f"adata shape: {adata.shape}")
        if adata.X.shape[0] == 0 or adata.X.shape[1] == 0:
            print(f"Warning: No gene expression data available for {donor_id}. Skipping...")
            continue  # Skip this donor and proceed to the next one

        # Preprocessing steps (normalization, log transformation, etc.)
        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)

        # Identify highly variable genes (HVGs)
        sc.pp.highly_variable_genes(adata, min_mean=0.1, max_mean=10, min_disp=0.5)
        adata = adata[:, adata.var['highly_variable']]

        # Plot the highly variable genes
        sc.pl.highly_variable_genes(adata, show=True)

        # Perform analysis specific to this donor here (e.g., differential expression, UMAP, etc.)
        # Example of UMAP:
        sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
        sc.tl.umap(adata)
        sc.pl.umap(adata, color=['donor_age_group', 'donor_age'], title=f'UMAP for {donor_id}')

        # Clear the data from memory
        del adata
    else:
        print(f"File {h5ad_file} does not exist. Skipping...")

    # Process .tsv file (if needed)
    tsv_file = os.path.join(tsv_directory, f'{donor_id}.donor.tsv')
    if os.path.exists(tsv_file):
        df = pd.read_csv(tsv_file, sep='\t')

        # Add donor age and age group to the DataFrame
        donor_info = donor_metadata.get(donor_id, {'age_group': 'Unknown', 'age': None})
        df['donor_age_group'] = donor_info['age_group']
        df['donor_age'] = donor_info['age']

        # Display the first few rows of the DataFrame with the new 'donor_age' and 'donor_age_group' columns
        print(df.head())

        # Clear the DataFrame from memory
        del df
    else:
        print(f"TSV file {tsv_file} does not exist. Skipping...")

    print(f"Finished processing {donor_id}.\n")

Processing HRX138216...


MemoryError: Unable to allocate 111. MiB for an array with shape (29204368,) and data type float32